# Apriori Algorithm Implementation Assignment

### Objective:
You will implement the **Apriori algorithm** from scratch (i.e., without using any libraries like `mlxtend`) to find frequent itemsets and generate association rules.

### Dataset:
Use the [Online Retail Dataset](https://www.kaggle.com/datasets/vijayuv/onlineretail) from Kaggle. You can filter it for a specific country (e.g., `United Kingdom`) and time range to reduce size if needed.

---

In [1]:
import pandas as pd

## Step 1: Data Preprocessing

- Load the dataset
- Remove rows with missing values
- Filter out rows where `Quantity <= 0`
- Convert Data into Basket Format

👉 **Implement code below**

In [2]:
# Load the dataset
df = pd.read_csv(r"C:\Ddrive data\Sem 5 Labs\Data Mining\ProjectTasks\Datasets\OnlineRetail.csv", encoding='ISO-8859-1')
# Preprocess as per the instructions above | We have already done in TASK 2

# Remove missing values
df.dropna(subset=["InvoiceNo", "Description"], inplace=True)

# Filter for Quantity <= 0
df = df[df["Quantity"] > 0]

# Filter for a specific country (United Kingdom)
df = df[df["Country"] == "United Kingdom"]

# for Basket
basket = df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

C:\Users\kunjp\AppData\Local\Temp\ipykernel_11956\2429339532.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(lambda x: 1 if x > 0 else 0)


## Step 2: Implement Apriori Algorithm
Step-by-Step Procedure:
1. Generate Frequent 1-Itemsets
Count the frequency (support) of each individual item in the dataset.
Keep only those with support ≥ min_support.
→ Result is L1 (frequent 1-itemsets)
2. Iterative Candidate Generation (k = 2 to n)
While L(k-1) is not empty:
a. Candidate Generation

Generate candidate itemsets Ck of size k from L(k-1) using the Apriori property:
Any (k-itemset) is only frequent if all of its (k−1)-subsets are frequent.
b. Prune Candidates
Eliminate candidates that have any (k−1)-subset not in L(k-1).
c. Count Support
For each transaction, count how many times each candidate in Ck appears.
d. Generate Frequent Itemsets
Form Lk by keeping candidates from Ck that meet the min_support.
Repeat until Lk becomes empty.
Implement the following functions:
1. `get_frequent_itemsets(transactions, min_support)` - Returns frequent itemsets and their support
2. `generate_candidates(prev_frequent_itemsets, k)` - Generates candidate itemsets of length `k`
3. `calculate_support(transactions, candidates)` - Calculates the support count for each candidate

**Write reusable functions** for each part of the algorithm.

In [ ]:
from math import ceil

# ---------------------------
# Helper (tiny, for readability)
# ---------------------------
def _min_count(min_support, n_tx):
    """
    Accept either absolute support (>=1) or relative support (0< s <=1).
    Returns the minimum support count as an integer.
    """
    return min_support if min_support >= 1 else ceil(min_support * n_tx)


# ------------------------------------------------------
# 1) calculate_support(transactions, candidates)
# ------------------------------------------------------
def calculate_support(transactions, candidates):
    """
    Count support *counts* for each candidate itemset.

    Parameters
    ----------
    transactions : list[set]
        Each transaction is a Python set of items.
    candidates : list[frozenset]
        Candidate itemsets to count.

    Returns
    -------
    dict[frozenset, int]
        Support *count* of each candidate.
    """
    # Initialize counts to zero for every candidate
    counts = {c: 0 for c in candidates}

    # For every transaction, increment counts of candidates contained in it
    for tx in transactions:
        for c in candidates:
            # c ⊆ tx ?  (Python: c.issubset(tx))
            if c.issubset(tx):
                counts[c] += 1

    return counts


# ------------------------------------------------------
# 2) generate_candidates(prev_frequent_itemsets, k)
# ------------------------------------------------------
def generate_candidates(prev_frequent_itemsets, k):
    """
    JOIN + PRUNE to build k-item candidates from (k-1)-item frequent sets,
    **without** using itertools.combinations.

    Parameters
    ----------
    prev_frequent_itemsets : iterable[frozenset]
        Frequent itemsets of size (k-1).
    k : int
        Target candidate size.

    Returns
    -------
    list[frozenset]
        Candidate k-itemsets that pass the Apriori prune.
    """
    prev_list = sorted([tuple(sorted(fs)) for fs in prev_frequent_itemsets])
    prev_set  = set(prev_frequent_itemsets)  # for O(1) membership tests

    candidates = []

    # --- JOIN step ---
    # Pairwise join itemsets that share the same first k-2 items.
    # Because prev_list is lexicographically sorted, we can compare prefixes.
    for i in range(len(prev_list)):
        for j in range(i + 1, len(prev_list)):
            # If prefixes differ, no further j will match (due to sorting)
            if prev_list[i][:k-2] != prev_list[j][:k-2]:
                break

            # Union the two (k-1)-item sets → at most k items
            union_set = frozenset(prev_list[i]) | frozenset(prev_list[j])

            # Keep only unions that actually grew to size k
            if len(union_set) != k:
                continue

            # --- PRUNE step (Apriori property) ---
            # Every (k-1)-subset of this k-candidate must be frequent.
            # Generate (k-1)-subsets *without* combinations:
            # remove each element once and check membership.
            all_subsets_frequent = True
            for x in union_set:
                if frozenset(union_set - {x}) not in prev_set:
                    all_subsets_frequent = False
                    break

            if all_subsets_frequent and union_set not in candidates:
                candidates.append(union_set)

    return candidates


# ------------------------------------------------------
# 3) get_frequent_itemsets(transactions, min_support)
# ------------------------------------------------------
def get_frequent_itemsets(transactions, min_support):
    """
    Full Apriori loop:
      - Build L1 (frequent 1-itemsets)
      - For k = 2.., iteratively create Ck (candidates), count support, prune to Lk
      - Stop when no more frequent itemsets

    Parameters
    ----------
    transactions : list[iterable]
        Raw transactions; each is an iterable of items. (Will be converted to sets.)
    min_support : float|int
        If >=1: absolute count. If 0<value<=1: relative min support (fraction).

    Returns
    -------
    dict[frozenset, int]
        All frequent itemsets with their support *counts*.
    """
    # Normalize transactions as list[set]
    tx = [set(t) for t in transactions]
    n_tx = len(tx)
    min_cnt = _min_count(min_support, n_tx)

    frequent = {}  # accumulate all Lk across levels

    # ---------- L1: frequent 1-itemsets ----------
    # Count singletons
    singleton_counts = {}
    for t in tx:
        for item in t:
            singleton_counts[item] = singleton_counts.get(item, 0) + 1

    Lk = {frozenset([i]): c for i, c in singleton_counts.items() if c >= min_cnt}
    frequent.update(Lk)

    k = 2
    # ---------- Iterate for k >= 2 ----------
    while Lk:
        # a) Candidate generation: Ck from L(k-1)
        Ck = generate_candidates(Lk.keys(), k)

        if not Ck:
            break

        # b) Count support for candidates
        Ck_counts = calculate_support(tx, Ck)

        # c) Prune to Lk (keep those that meet min support)
        Lk = {c: cnt for c, cnt in Ck_counts.items() if cnt >= min_cnt}

        # d) Accumulate and move to next level
        frequent.update(Lk)
        k += 1

    return frequent


## Step 3: Generate Association Rules

- Use frequent itemsets to generate association rules
- For each rule `A => B`, calculate:
  - **Support**
  - **Confidence**
- Only return rules that meet a minimum confidence threshold (e.g., 0.5)

👉 **Implement rule generation function below**

## Step 4: Output and Visualize

- Print top 10 frequent itemsets
- Print top 10 association rules (by confidence or lift)

👉 **Output results below**